# Dataset preparation - SameSentiment Yelp - Base

In [ ]:
import pickle
from pathlib import Path

from tqdm import tqdm
from transformers.trainer_utils import set_seed

tqdm.pandas()

In [ ]:
# download + scp to server + extract
data_yelp_path = Path("data/sentiment/yelp/")

# ------------------------------------

# local?
data_yelp_path = Path("data_raw/sentiment/yelp/")

# local? - output path (base) for sentiment review yelp pairs
data_yelp_b_tdt_path = Path("data/sentiment/yelp-pair-b/")
data_yelp_b_rand_tdt_path = Path("data/sentiment/yelp-pair-rand-b/")
# local - output path for simple sentiment reviews yelp
data_yelp_tdt_sentiment_5_path = Path("data/sentiment/yelp-sentiment-5/")
data_yelp_tdt_sentiment_b_path = Path("data/sentiment/yelp-sentiment-b/")

---

In [ ]:
dn_yelp_cached = data_yelp_path / "cached"

In [ ]:
#  #### Load categories & topics
from data_prep import load_reviews, load_topics

# ##### Filter categories
from data_prep import filter_min_cat_combis, make_map_cats, make_cat_combis

# ##### Filter reviews
from data_prep import filter_min_review_freq, filter_both_good_bad

# ##### Filter businesses
from data_prep import filter_by_businesses, filter_by_businesses_not_same

# #### Load category tree
from data_prep import load_category_tree
from data_prep import get_root_category_items, get_children_category_item_list
from data_prep import get_businesses_in_category, get_businesses_in_category_branch


# #### Cache root category reviews in dataframes
from data_prep import cache_root_category_businesses_df, load_cached_root_category_businesses_df


# #### Positive + negative same-sentiment pairs
from data_prep import make_pairs_good_bad
from data_prep import make_pairs_good_bad_over_business

# #### Not same-sentiment pairs (combinations positive + negative)
from data_prep import make_pairs_negative
from data_prep import make_pairs_negative_over_business

# #### Dataframe for training etc.
from data_prep import make_or_load_pairs
from data_prep import make_or_load_pairs_over_businesses


# #### Make train/dev/test splits
from data_prep import split_df, write_pair_df_tsv, write_pair_tdt_tsv


---

In [ ]:
# N positive + N negative
# --> 2N pos+neg (not same-sentiment)
num_pairs_per_class = 2

#: number of negative same-sentiment samples same as positive same-sentiment samples
num_pairs_negative = 2 * num_pairs_per_class

#: whether for a single side (good or bad) there can be multiple occurrences of the same review
#: may need to check afterwared that not by chance same pairing happens ...
repeatable_on_side = False

---

## Write out training data

### Split Test-Data

In [ ]:
fn_yelp_df = data_yelp_path / "df_traindev4_typed.p"

with open(fn_yelp_df, "rb") as fp:
    traindev_df = pickle.load(fp)

In [ ]:
fn_yelp_df = data_yelp_path / "df_traindev_test.p"
#fn_yelp_df = data_yelp_path / "df_traindev_test_over_business.p"
#traindev_df = df_traindev

In [ ]:
# store
traindev_df, test_df = split_df(traindev_df, ratio=0.1, do_shuffle=True, random_state=42, name_train="traindev", name_dev="test")

with open(fn_yelp_df, "wb") as fp:
    pickle.dump(traindev_df, fp, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(test_df, fp, protocol=pickle.HIGHEST_PROTOCOL)

---

In [ ]:
with open(fn_yelp_df, "rb") as fp:
    traindev_df = pickle.load(fp)
    test_df = pickle.load(fp)

In [ ]:
root_path = data_yelp_b_tdt_path
#root_path = data_yelp_b_rand_tdt_path

In [ ]:
write_pair_tdt_tsv(root_path, traindev_df, split_test=0.1, split_dev=0.3)

In [ ]:
# symlink pred.tsv
! ln -s test.tsv {root_path}/pred.tsv

In [ ]:
print("@ ", root_path, "\n")
! ls -lh {root_path}

In [ ]:
#model_name = "bert-base-uncased"
#model_name = "bert-base-cased"
model_name = "distilroberta-base"
#model_name = "distilbert-base-cased"
#model_name = "albert-base-v2"

data_name = "yelp-pair-b"
#data_name = "yelp-pair-rand-b" ## over businesses

seq_len = 256
batch_size = 16
acc_steps = 64
num_epoch = 3
cuda_devs = "1"

run_name = f"{model_name.replace('/', '-')}-{data_name}_{seq_len}_{batch_size}-acc{acc_steps}_{num_epoch}"

In [ ]:
# create folder for logging
! mkdir -p ./output_sent_logs/{run_name}

! \
    MLFLOW_EXPERIMENT_NAME=same-sentiment \
    CUDA_VISIBLE_DEVICES={cuda_devs} \
    python trainer.py \
    --do_train --do_eval --do_test \
    --model_name_or_path {model_name} \
    --task_name same-b \
    --data_dir ./data/sentiment/{data_name} \
    --output_dir ./output_sent/{run_name} \
    --run_name {run_name} \
    --per_device_eval_batch_size {batch_size} \
    --per_device_train_batch_size {batch_size} \
    --gradient_accumulation_steps {acc_steps} \
    --logging_steps 5000 \
    --save_steps 10000 \
    --save_total_limit 3 \
    --num_train_epochs {num_epoch} \
    --max_seq_length {seq_len} \
    --evaluation_strategy epoch \
    > >(tee -a ./output_sent_logs/{run_name}/stdout.log) \
    2> >(tee -a ./output_sent_logs/{run_name}/stderr.log >&2)

# --overwrite_output_dir \
# --overwrite_cache \
# --eval_steps 128 \
# --evaluation_strategy steps \
# --load_best_model_at_end \
# HF_MLFLOW_LOG_ARTIFACTS=TRUE \

Train ...

```bash
CUDA_VISIBLE_DEVICES=1 python trainer.py --do_train --do_eval --model_name_or_path bert-base-uncased --task_name same-b --data_dir ./data/sentiment/yelp-pair-b --output_dir ./output/yelp-pair-b_128_32_3 --run_name yelp-pair-b_128_32_3 --per_device_eval_batch_size 32 --per_device_train_batch_size 32 --logging_steps 10000 --save_steps 2000 --num_train_epochs 3 --max_seq_length 128
```

Test ...

```bash
CUDA_VISIBLE_DEVICES=1 python trainer.py --do_test --model_name_or_path bert-base-uncased --task_name same-b --data_dir ./data/sentiment/yelp-pair-b --output_dir ./output/yelp-pair-b_128_32_3 --run_name yelp-pair-b_128_32_3 --per_device_eval_batch_size 32 --logging_steps 10000 --max_seq_length 128
```

In [ ]:
! CUDA_VISIBLE_DEVICES=0 python trainer.py --do_eval --do_test --model_name_or_path bert-base-uncased --task_name same-b --data_dir ./data/sentiment/yelp-pair-b --output_dir ./output/yelp-pair-b_128_32_3 --run_name yelp-pair-b_128_32_3 --per_device_eval_batch_size 32 --logging_steps 10000 --max_seq_length 128

Model: **distilroberta-base**

```bash
CUDA_VISIBLE_DEVICES=0 python trainer.py --do_train --do_eval --do_test --model_name_or_path distilroberta-base --task_name same-b --data_dir ./data/sentiment/yelp-pair-b --output_dir ./output/distilroberta-base-yelp-pair-b_128_32_3 --run_name distilroberta-base-yelp-pair-b_128_32_3 --per_device_eval_batch_size 32 --per_device_train_batch_size 32 --logging_steps 10000 --save_steps 2000 --num_train_epochs 3 --max_seq_length 128
```